In [ ]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="whitegrid")
import matplotlib.ticker as tick
from matplotlib.ticker import PercentFormatter

In [ ]:
train_I = pd.read_csv('/kaggle/input/ieee-fraud-detection/train_identity.csv')
train_T = pd.read_csv('/kaggle/input/ieee-fraud-detection/train_transaction.csv')

In [ ]:
mergedset = train_T.merge(train_I, how= 'left')

# Parte I: Análisis exploratorio

Realizar 6 visualizaciones interesantes que ayuden a explicar el target, alguno que
contenga al menos un feature de train_identity.csv y haciendo al menos un plot de cada tipo:
*  Bar plot
*  Violin plot
*  Box plot
*  Heatmap


## Heatmap de correlacion 

In [ ]:
corrmatrix = mergedset.corr()[['isFraud']]

fig, ax = plt.subplots(1,2,figsize=(14, 10))
maximos = corrmatrix.loc[(corrmatrix['isFraud']>0) & (corrmatrix['isFraud']!=1)].sort_values(by='isFraud', ascending=False).head(20)
heatmapmax = sns.heatmap(maximos, vmin=maximos.min(), vmax=maximos.max(), annot=True, cmap=sns.light_palette("seagreen", as_cmap=True),ax = ax[0])
heatmapmax.set_title('Features mas relacionadas con isFraud', fontdict={'fontsize':18}, pad=16);

minimos = corrmatrix.loc[corrmatrix['isFraud']<0].sort_values(by='isFraud', ascending=True).head(20)
heatmapmin = sns.heatmap(minimos, vmin=minimos.max(), vmax=minimos.min(), annot=True, cmap=sns.color_palette("light:b", as_cmap=True),ax = ax[1])
heatmapmin.set_title('Features menos relacionadas con isFraud', fontdict={'fontsize':18}, pad=16);
fig.tight_layout(pad=3.0)

## Violin plot de fraude por Browser

In [ ]:
mergedset['id_31'].fillna('none',inplace=True)
mergedset.loc[mergedset['id_31'].str.lower().str.contains("chrome"),['Browser']] = 'Chrome'
mergedset.loc[mergedset['id_31'].str.lower().str.contains("samsung"),['Browser']] = 'Samsung'
mergedset.loc[mergedset['id_31'].str.lower().str.contains("ie"),['Browser']] = 'ie'
mergedset.loc[mergedset['id_31'].str.lower().str.contains("safari"),['Browser']] = 'Safari'
mergedset.loc[mergedset['id_31'].str.lower().str.contains("firefox"),['Browser']] = 'Firefox'
mergedset.loc[mergedset['id_31'].str.lower().str.contains("opera"),['Browser']] = 'Opera'
mergedset.loc[mergedset['id_31'].str.lower().str.contains("edge"),['Browser']] = 'Edge'
mergedset.loc[mergedset['id_31'].str.lower().str.contains("none"),['Browser']] = 'None'
mergedset.loc[(mergedset['Browser'].isna()) & (~mergedset['id_31'].isna()),['Browser']] = 'Other'

browserinfo = mergedset.loc[(mergedset['Browser'] != 'None') & (mergedset['isFraud'] == 1),['Browser','TransactionAmt']]
fig, ax = plt.subplots(1,1,figsize=(20, 5))
sns.violinplot(x=browserinfo.Browser, y=browserinfo.TransactionAmt, data=browserinfo, width=0.9, fliersize = 3,palette= sns.color_palette("Paired"),ax=ax)
ax.set_xlabel('Browser')
ax.set_title('Distribucion de montos de los fraudes por Browser', fontsize=16, pad=20)
ax.set_ylabel('Monto')

## Boxplot de dias que pasaron desde el fraude por tarjeta

In [ ]:
deltainfo = mergedset.loc[( mergedset['isFraud'] == 1),['card4','TransactionDT']]
deltainfo['days'] = deltainfo['TransactionDT'] / 86400
fig, ax = plt.subplots(1,1,figsize=(20, 5))
sns.boxplot(x=deltainfo.card4, y=deltainfo.days, data=deltainfo, width=0.9, fliersize = 3,palette= sns.color_palette("Paired"),ax=ax)
ax.set_xlabel('Tarjeta')
ax.set_title('Distribucion de cantidad de dias desde el fraude por tarjeta', fontsize=16, pad=20)
ax.set_ylabel('Dias')
fig.tight_layout(pad=3.0)

## Heatmap de relacion entre productos y tarjeta para los fraudes

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(10, 5))

gDE = sns.heatmap(pd.pivot_table(mergedset.loc[:,['ProductCD','card4','isFraud']], values='isFraud', index=['card4'],
                    columns=['ProductCD'], aggfunc=np.sum).fillna(0), cmap=sns.color_palette("YlOrBr", as_cmap=True), annot = True,fmt='1.0f',ax = ax)
gDE.set_title("Relacion entre product code y tarjetas para fraudes", fontsize = 20,pad = 20)
gDE.set_xlabel("Codigo de producto", fontsize = 15)
gDE.set_ylabel("Tarjeta", fontsize = 15)
plt.yticks(rotation=0)
plt.xticks(rotation=0)

fig.tight_layout(pad=3.0)

## Barplot de la cantidad de fraudes por dispositivo y browser

In [ ]:
mergedset['id_31'].fillna('none',inplace=True)
mergedset.loc[mergedset['id_31'].str.lower().str.contains("chrome"),['Browser']] = 'Chrome'
mergedset.loc[mergedset['id_31'].str.lower().str.contains("samsung"),['Browser']] = 'Samsung'
mergedset.loc[mergedset['id_31'].str.lower().str.contains("ie"),['Browser']] = 'ie'
mergedset.loc[mergedset['id_31'].str.lower().str.contains("safari"),['Browser']] = 'Safari'
mergedset.loc[mergedset['id_31'].str.lower().str.contains("firefox"),['Browser']] = 'Firefox'
mergedset.loc[mergedset['id_31'].str.lower().str.contains("opera"),['Browser']] = 'Opera'
mergedset.loc[mergedset['id_31'].str.lower().str.contains("edge"),['Browser']] = 'Edge'
mergedset.loc[mergedset['id_31'].str.lower().str.contains("none"),['Browser']] = 'None'
mergedset.loc[(mergedset['Browser'].isna()) & (~mergedset['id_31'].isna()),['Browser']] = 'Other'

deviceinfo = pd.pivot_table(mergedset.loc[:,['Browser','DeviceType','isFraud']], index = ['Browser'],values='isFraud', columns = ['DeviceType'], aggfunc = pd.Series.sum).fillna('0')
deviceinfo.columns = ['desktop','mobile']
deviceinfo = deviceinfo.loc[('Chrome','Edge','Firefox','Opera','Safari','Samsung','ie','Other'),:]
for col in deviceinfo.columns:
    deviceinfo[col] = pd.to_numeric(deviceinfo[col])#

deviceinfo.plot(kind = 'bar', stacked = False, colormap = 'tab20', width = 0.8, grid = False,figsize= (20,10))
plt.legend(loc = 'center right', bbox_to_anchor = (1.2, 0.5))
plt.xlabel('Browser', fontsize = 15)
plt.ylabel('Fraudes', fontsize = 15)
plt.title('Cantidad de fraudes por browser y tipo de dispositivo', fontsize=20)
plt.xticks(rotation = 0)
fig.tight_layout(pad=3.0)

## Barplot de la cantidad de fraudes por dispositivo y marca

In [ ]:
mergedset['DeviceInfo'].fillna('none',inplace=True)
mergedset.loc[mergedset['DeviceInfo'].str.lower().str.contains("windows"),['Device']] = 'WINDOWS'
mergedset.loc[mergedset['DeviceInfo'].str.lower().str.contains("ios"),['Device']] = 'IOS'
mergedset.loc[mergedset['DeviceInfo'].str.lower().str.contains("mac"),['Device']] = 'MAC'
mergedset.loc[(mergedset['DeviceInfo'].str.lower().str.contains("gt-")) | (mergedset['DeviceInfo'].str.lower().str.contains("sm-")) | (mergedset['DeviceInfo'].str.lower().str.contains("samsung")),['Device']] = 'SAMSUNG'
mergedset.loc[mergedset['DeviceInfo'].str.lower().str.contains("lg"),['Device']] = 'LG'
mergedset.loc[mergedset['DeviceInfo'].str.lower().str.contains("htc"),['Device']] = 'HTC'
mergedset.loc[(mergedset['DeviceInfo'].str.lower().str.contains("xt")) | (mergedset['DeviceInfo'].str.lower().str.contains("moto")),['Device']] = 'MOTO'
mergedset.loc[mergedset['DeviceInfo'].str.lower().str.contains("hisense"),['Device']] = 'HISENSE'
mergedset.loc[mergedset['DeviceInfo'].str.lower().str.contains("huawei"),['Device']] = 'HUAWEI'
mergedset.loc[mergedset['DeviceInfo'].str.lower().str.contains("lenovo"),['Device']] = 'LENOVO'
mergedset.loc[mergedset['DeviceInfo'].str.lower().str.contains("pixel"),['Device']] = 'PIXEL'
mergedset.loc[mergedset['DeviceInfo'].str.lower().str.contains("redmi"),['Device']] = 'REDMI'
mergedset.loc[mergedset['DeviceInfo'].str.lower().str.contains("blade"),['Device']] = 'BLADE'
mergedset.loc[mergedset['DeviceInfo'].str.lower().str.contains("nexus"),['Device']] = 'NEXUS' 
mergedset.loc[mergedset['DeviceInfo'].str.lower().str.contains("trident"),['Device']] = 'TRIDENT' 
mergedset.loc[mergedset['DeviceInfo'].str.lower().str.contains("none"),['Device']] = 'NO INFO'
mergedset.loc[(mergedset['Device'].isna()) & (~mergedset['DeviceInfo'].isna()),['Device']] = 'OTHER'


deviceinfo = pd.pivot_table(mergedset.loc[:,['Device','DeviceType','isFraud']], index = ['Device'],values='isFraud', columns = ['DeviceType'], aggfunc = pd.Series.sum).fillna('0')
deviceinfo.columns = ['desktop','mobile']
deviceinfo = deviceinfo.loc[('BLADE', 'HISENSE', 'HTC', 'HUAWEI', 'IOS', 'LENOVO', 'LG', 'MAC',
       'MOTO', 'NEXUS', 'NO INFO', 'OTHER', 'PIXEL', 'REDMI', 'SAMSUNG',
       'TRIDENT', 'WINDOWS'),:]
for col in deviceinfo.columns:
    deviceinfo[col] = pd.to_numeric(deviceinfo[col])#

deviceinfo.plot(kind = 'bar', stacked = False, colormap = 'tab20', width = 0.8, grid = False,figsize= (20,10))
plt.legend(loc = 'center right', bbox_to_anchor = (1.2, 0.5))
plt.xlabel('Marca', fontsize = 15)
plt.ylabel('Fraudes', fontsize = 15)
plt.title('Cantidad de fraudes por marca y tipo de dispositivo', fontsize=20)
plt.xticks(rotation = 0)
